# [ HW_2 ] 
# Titanic_dataset

- Train (훈련 데이터): 이 데이터 집합은 모델을 훈련시키는 데 사용됩니다. 모델은 이 데이터를 기반으로 학습하고 패턴을 파악합니다.

- Validation (검증 데이터): 이 데이터 집합은 모델의 성능을 평가하고 하이퍼파라미터 조정에 사용됩니다. 모델은 검증 데이터를 사용하여 훈련 중에 얼마나 잘 수행되고 있는지를 확인합니다.

- Test (테스트 데이터): 이 데이터 집합은 모델의 최종 성능을 평가하기 위해 사용됩니다. 모델이 새로운, 이전에 본 적이 없는 데이터에 얼마나 잘 일반화되는지를 평가하는 데 사용됩니다.

In [124]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split

In [125]:
class TitanicDataset(Dataset):
    # 클래스의 생성자 매서드, X y를 텐서로 변환하여 저장
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)
        self.y = torch.LongTensor(y)

    def __len__(self):
        return len(self.X)

    # idx에 해당하는 데이터 포인트(Titanic 데이터셋의 데이터 포인트 알아내기)를 반환한다.
    # 각 데이터 포인트는 'input'과 'target'키를 가지는 딕셔너리로 표현된다.
    def __getitem__(self, idx):
        feature = self.X[idx]
        target = self.y[idx]
        return {'input': feature, 'target': target}

    def __str__(self):
        str = "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
          len(self.X), self.X.shape, self.y.shape
        )
        return str

In [126]:
class TitanicTestDataset(Dataset):
    def __init__(self, X):
        self.X = torch.FloatTensor(X)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        feature = self.X[idx]
        return {'input': feature}

    def __str__(self):
        str = "Data Size: {0}, Input Shape: {1}".format(
          len(self.X), self.X.shape
        )
        return str

In [127]:
# Titanic 데이터셋에 대한 전처리 및 데이터 준비 단계를 수행
def get_preprocessed_dataset():
    # CURRENT_FILE_PATH : 현재 스크립트 파일의 디렉토리 경로를 저장
    ###CURRENT_FILE_PATH = os.path.dirname(os.path.abspath(__file__))
    CURRENT_FILE_PATH = "C:/Users/joowo/deep_learning/HW2/CSV"
    ###print(CURRENT_FILE_PATH)

    # 훈련데이터와 테스트 데이터 파일 경로를 구성
    # os.path.join을 사용하여 현재 스크립트 파일의 디렉토리와 파일 이름을 결합하여 절대 경로를 얻음
    train_data_path = os.path.join(CURRENT_FILE_PATH, "train.csv")
    test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv")

    # 각 데이터를 데이터프레임으로 읽어옴
    # pandas 데이터프레임 -> 데이터를 표 형태로 처리해줌
    train_df = pd.read_csv(train_data_path)
    test_df = pd.read_csv(test_data_path)
    
    
    # all_df는 훈련 데이터와 테스트 데이터를 연결하여 하나의 데이터프레임으로 합침
    # pd.concat: pandas 라이브러리에서 제공하는 데이터프레임을 결합(연결)하는 함수
    # <매개변수들>
    # objs: 결합하려는 데이터프레임들을 리스트로 나타내며, 이것이 결합할 대상임
    # axis: 데이터프레임을 어느 축을 기준으로 결합할 것인지를 나타내며, 0은 행(수직 결합), 1은 열(수평 결합)을 의미
    # 기타 매개변수: ignore_index, keys, verify_integrity, sort 등 다양한 매개변수가 있어 데이터 결합 방식을 제어하는 데 사용됩니다.
    all_df = pd.concat([train_df, test_df], sort=False) # sort=False 면 데이터를 결합할 때 열의 순서를 정렬하지 않음

    # 데이터프레임 전처리
    all_df = get_preprocessed_dataset_1(all_df)

    all_df = get_preprocessed_dataset_2(all_df)

    all_df = get_preprocessed_dataset_3(all_df)

    all_df = get_preprocessed_dataset_4(all_df)

    all_df = get_preprocessed_dataset_5(all_df)

    all_df = get_preprocessed_dataset_6(all_df)

    train_X = all_df[~all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)
    train_y = train_df["Survived"]

    test_X = all_df[all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)

    #  훈련 데이터셋 객체(dataset)를 생성하고 초기화
    dataset = TitanicDataset(train_X.values, train_y.values)
    ###print(dataset)
    
    # random_split 함수를 사용하여 훈련 데이터셋을 훈련 데이터셋과 검증 데이터셋으로 나눔
    train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2])
    
    # 테스트 데이터셋 객체(test_dataset)를 생성하고 초기화
    test_dataset = TitanicTestDataset(test_X.values)
    ###print(test_dataset)

    return train_dataset, validation_dataset, test_dataset

In [128]:
def get_preprocessed_dataset_1(all_df):
    # Pclass별 Fare 평균값을 사용하여 Fare 결측치 메우기
    Fare_mean = all_df[["Pclass", "Fare"]].groupby("Pclass").mean().reset_index()
    Fare_mean.columns = ["Pclass", "Fare_mean"]
    all_df = pd.merge(all_df, Fare_mean, on="Pclass", how="left")
    all_df.loc[(all_df["Fare"].isnull()), "Fare"] = all_df["Fare_mean"]

    return all_df


def get_preprocessed_dataset_2(all_df):
    # name을 세 개의 컬럼으로 분리하여 다시 all_df에 합침
    name_df = all_df["Name"].str.split("[,.]", n=2, expand=True)
    name_df.columns = ["family_name", "honorific", "name"]
    name_df["family_name"] = name_df["family_name"].str.strip()
    name_df["honorific"] = name_df["honorific"].str.strip()
    name_df["name"] = name_df["name"].str.strip()
    all_df = pd.concat([all_df, name_df], axis=1)

    return all_df


def get_preprocessed_dataset_3(all_df):
    # honorific별 Age 평균값을 사용하여 Age 결측치 메우기
    honorific_age_mean = all_df[["honorific", "Age"]].groupby("honorific").median().round().reset_index()
    honorific_age_mean.columns = ["honorific", "honorific_age_mean", ]
    all_df = pd.merge(all_df, honorific_age_mean, on="honorific", how="left")
    all_df.loc[(all_df["Age"].isnull()), "Age"] = all_df["honorific_age_mean"]
    all_df = all_df.drop(["honorific_age_mean"], axis=1)

    return all_df


def get_preprocessed_dataset_4(all_df):
    # 가족수(family_num) 컬럼 새롭게 추가
    all_df["family_num"] = all_df["Parch"] + all_df["SibSp"]

    # 혼자탑승(alone) 컬럼 새롭게 추가
    all_df.loc[all_df["family_num"] == 0, "alone"] = 1
    all_df["alone"].fillna(0, inplace=True)

    # 학습에 불필요한 컬럼 제거
    all_df = all_df.drop(["PassengerId", "Name", "family_name", "name", "Ticket", "Cabin"], axis=1)

    return all_df


def get_preprocessed_dataset_5(all_df):
    # honorific 값 개수 줄이기
    all_df.loc[
    ~(
            (all_df["honorific"] == "Mr") |
            (all_df["honorific"] == "Miss") |
            (all_df["honorific"] == "Mrs") |
            (all_df["honorific"] == "Master")
    ),
    "honorific"
    ] = "other"
    all_df["Embarked"].fillna("missing", inplace=True)

    return all_df


def get_preprocessed_dataset_6(all_df):
    # 카테고리 변수를 LabelEncoder를 사용하여 수치값으로 변경하기
    category_features = all_df.columns[all_df.dtypes == "object"]
    from sklearn.preprocessing import LabelEncoder
    for category_feature in category_features:
        le = LabelEncoder()
        if all_df[category_feature].dtypes == "object":
            le = le.fit(all_df[category_feature])
            all_df[category_feature] = le.transform(all_df[category_feature])

    return all_df

### 데이터 전처리 1차
* Pclass 별로 평균 Fare 값을 계산하고, 이 값을 사용하여 Fare 열의 결측치를 메웁니다.

### 데이터 전처리 2차
* Name 열을 콤마와 점을 기준으로 세 부분("family_name," "honorific," "name")으로 나누어 데이터프레임에 추가

### 데이터 전처리 3차
* honorific 별로 평균 Age 값을 계산하고, 이 값을 사용하여 Age 열의 결측치를 메움

### 데이터 전처리 4차
* Parch (부모/자녀 수)와 SibSp (형제자매/배우자 수)를 합산하여 family_num 열을 새로 추가함
* 혼자 탑승 여부를 나타내는 alone 열을 추가하며, 가족이 없는 승객은 1로 표시되고, 가족이 있는 승객은 0으로 표시됨
* 학습에 불필요한 열들을 제거하고 필요한 특성을 생성

### 데이터 전처리 5차
* honorific 열의 값 중 Mr, Miss, Mrs, Master 이외의 값들을 "other"로 대체
* Embarked 열의 결측치를 "missing"으로 대체

### 데이터 전처리 6차
* 카테고리 변수를 숫자 형태로 변환하기 위해 LabelEncoder를 사용
* 모든 열 중에서 데이터 타입이 "object" (문자열)인 열들을 선택하고, 각각의 열에 대해 LabelEncoder를 적용하여 문자열 값을 숫자로 변환

# Titanic Deep Learning Model Training

In [129]:
import wandb
from torch import nn
from datetime import datetime

In [130]:
wandb.login()

True

In [131]:
from torch import nn, optim

# 신경망 모델을 정의하는 클래스인 MyModel을 생성
class MyModel(nn.Module):
    # 입력 차원 (n_input)과 출력 차원 (n_output)에 따라 다층 퍼셉트론 (MLP) 모델을 정의
    def __init__(self, n_input, n_output):
        super().__init__()

        self.model = nn.Sequential(
            nn.Linear(n_input, 30), # 입력 레이어: nn.Linear(n_input, 30)은 입력 차원 n_input과 출력 차원 30을 갖는 선형 레이어를 생성
            nn.RReLU(), # ReLU 활성화 함수: 비선형성을 추가
            nn.Linear(30, 30), # 은닉 레이어: 30차원 입력에서 다시 30차원 출력을 생성
            nn.RReLU(), # ReLU 활성화 함수: 다시 비선형성을 추가
            nn.Linear(30, 30), # 은닉 레이어: 30차원 입력에서 다시 30차원 출력을 생성
            nn.RReLU(), # ReLU 활성화 함수: 다시 비선형성을 추가
            nn.Linear(30, 30), # 은닉 레이어: 30차원 입력에서 다시 30차원 출력을 생성
            nn.RReLU(), # ReLU 활성화 함수: 다시 비선형성을 추가
            nn.Linear(30, 30), # 은닉 레이어: 30차원 입력에서 다시 30차원 출력을 생성
            nn.RReLU(), # ReLU 활성화 함수: 다시 비선형성을 추가
            nn.Linear(30, n_output), # 출력 레이어: 마지막으로 30차원 입력에서 출력 차원 n_output의 결과를 생성
            )
    
    # 입력 x에 대해 모델의 순전파 연산을 수행
    def forward(self, x):
        x = self.model(x)
        return x

#### RReLU()
* RReLU는 음수 입력 값에 대해서 랜덤하게 생성된 양수 기울기를 사용함 
* RReLU는 일부 뉴런이 활성화되지 않는 문제를 완화하고, 모델의 일반화 능력을 향상시킬 수 있음

#### MyModel
* 입력층 : 입력 n_input, 출력 30
* 은닉층1 : 입력 30, 출력 30
* 은닉층2 : 입력 30, 출력 30
* 은닉층3 : 입력 30, 출력 30
* 은닉층4 : 입력 30, 출력 30
* 출력층 : 입력 30, 출력 n_output

In [132]:
def test(test_data_loader): # test_data_loader: 테스트 데이터셋을 미니배치로 제공하는 역할, 미니배치: 학습 데이터의 부분 집합을 의미
    print("[TEST]")
    
    # next(iter(test_data_loader))로 첫 번째 미니배치를 가져옴. 
    # 이 미니배치는 딕셔너리 형태로 구성되어 있으며, 'input' 키를 사용하여 입력 데이터를 추출
    batch = next(iter(test_data_loader))
    print("{0}".format(batch['input'].shape))
    my_model = MyModel(n_input=11, n_output=2) # 입력데이터 11차원, 출력데이터 2차원
    
    output_batch = my_model(batch['input'])
    prediction_batch = torch.argmax(output_batch, dim=1)
    for idx, prediction in enumerate(prediction_batch, start=892):
        print(idx, prediction.item())

In [133]:
if __name__ == "__main__":
    ENV_NAME = "Titanic_DL_Model_Training" # 프로젝트 이름
    use_wandb = True
    current_time = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')
    config = {
        "env_name": ENV_NAME,
        "max_num_epoch": 100,
        "batch_size": 16,
        "learning_rate": 0.001,
    }
    
    if use_wandb:
            wandb = wandb.init(
                project=config["env_name"],
                name=current_time,
                config=config
            )
    
    train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()

    print("train_dataset: {0}, validation_dataset.shape: {1}, test_dataset: {2}".format(
        len(train_dataset), len(validation_dataset), len(test_dataset)
    ))
    print("#" * 50, 1)

    for idx, sample in enumerate(train_dataset):
        # 
        print("{0} - {1}: {2}".format(idx, sample['input'], sample['target']))

    print("#" * 50, 2)

    train_data_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
    validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=16, shuffle=True)
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

    print("[TRAIN]")
    for idx, batch in enumerate(train_data_loader):
        print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))
        
    print("[VALIDATION]")
    for idx, batch in enumerate(validation_data_loader):
        print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

    print("#" * 50, 3)

    test(test_data_loader)
        
    my_model = MyModel(n_input=11, n_output=2)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(my_model.parameters(), lr=config["learning_rate"])

    # 설정된 에포크 수 동안 반복하여 모델을 학습
    for epoch in range(config["max_num_epoch"]):
        # 모델을 학습모드로 설정. 이렇게 하면 모델 내의 레이어가 학습중에 파라미터 업데이트를 수행하게 됨
        my_model.train() 
        total_loss = 0.0
        correct = 0
        total = 0

        # train_data_loader를 사용하여 학습 데이터를 미니배치로 나누어 학습을 수행
        # enumerate(train_data_loader)를 통해 각 미니배치와 해당 인덱스를 순회
        for idx, batch in enumerate(train_data_loader):
            optimizer.zero_grad() # 이전 미니배치에서의 그래디언트 초기화
            input_data = batch['input']
            target = batch['target']

            output = my_model(input_data)
            loss = criterion(output, target)
            loss.backward() # 손실에 대한 그레디언트 계산
            optimizer.step() # 파라미터 업데이트. 역전파 알고리즘을 통해 네트워크의 가중치와 bias를 조정

            # 정확도와 손실을 기록
            
            total_loss += loss.item()

            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

        accuracy = correct / total
        average_loss = total_loss / len(train_data_loader)

        if use_wandb:
            # 학습 중에 획득한 손실(average_loss) 및 정확도(accuracy)를 기록
            wandb.log({"train_loss": average_loss, "train_accuracy": accuracy})

        # 학습 중에 현재 에포크 번호, 평균 손실, 정확도를 출력    
        print(f"Epoch [{epoch + 1}/{config['max_num_epoch']}], "
            f"Loss: {average_loss:.4f}, "
            f"Accuracy: {accuracy:.4f}")
        
        my_model.eval() # 모델을 평가모드로 설정. 모델의 가중치가 고정되고 역전파가 비활성화됨
        total_loss = 0.0
        correct = 0
        total = 0

        # with torch.no_grad() 블록 내에서 검증 데이터를 사용하여 모델의 성능을 평가
        with torch.no_grad():
            for idx, batch in enumerate(validation_data_loader):
                input_data = batch['input']
                target = batch['target']

                output = my_model(input_data)
                loss = criterion(output, target)

                total_loss += loss.item()

                _, predicted = torch.max(output.data, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()

        accuracy = correct / total
        average_loss = total_loss / len(validation_data_loader)

        if use_wandb:
            wandb.log({"val_loss": average_loss, "val_accuracy": accuracy})

        print(f"Validation Loss: {average_loss:.4f}, "
            f"Validation Accuracy: {accuracy:.4f}")
    wandb.finish()

        
    final_model_path = './my_model.pth'
    torch.save(my_model.state_dict(), final_model_path)

    my_model = MyModel(n_input=11, n_output=2)
    my_model.load_state_dict(torch.load('./my_model.pth'))
    my_model.eval()

    # 테스트 데이터를 DataLoader에 넣음
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

    # 모델을 사용하여 테스트 데이터에 대한 예측을 수행
    predictions = []
    for batch in test_data_loader:
        inputs = batch['input']
        outputs = my_model(inputs)
        predictions_batch = torch.argmax(outputs, dim=1)
        predictions.extend(predictions_batch.tolist())

    # submission.csv 파일을 생성
    submission_df = pd.DataFrame({
        'PassengerId': range(892, 892 + len(predictions)),
        'Survived': predictions
    })

    # CSV 파일로 저장
    submission_df.to_csv('./submission.csv', index=False)

    print("Submission 파일 'submission.csv'가 생성되었습니다.")

train_dataset: 713, validation_dataset.shape: 178, test_dataset: 418
################################################## 1
0 - tensor([ 3.0000,  1.0000, 29.0000,  0.0000,  0.0000,  7.7500,  1.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
1 - tensor([ 3.0000,  1.0000, 34.5000,  0.0000,  0.0000,  6.4375,  0.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
2 - tensor([ 1.0000,  1.0000, 52.0000,  0.0000,  0.0000, 30.5000,  2.0000, 87.5090,
         4.0000,  0.0000,  1.0000]): 1
3 - tensor([ 1.0000,  1.0000, 32.0000,  0.0000,  0.0000, 30.5000,  0.0000, 87.5090,
         4.0000,  0.0000,  1.0000]): 1
4 - tensor([ 3.0000,  0.0000, 26.0000,  1.0000,  0.0000, 16.1000,  2.0000, 13.3029,
         3.0000,  1.0000,  0.0000]): 0
5 - tensor([ 2.0000,  0.0000, 34.0000,  0.0000,  0.0000, 10.5000,  2.0000, 21.1792,
         3.0000,  0.0000,  1.0000]): 1
6 - tensor([ 3.0000,  0.0000, 22.0000,  0.0000,  0.0000,  7.7500,  1.0000, 13.3029,
         1.0000,  0.0000,  1.0000]): 1
7 - tensor([ 3.00

575 - tensor([ 2.0000,  0.0000, 29.0000,  1.0000,  0.0000, 26.0000,  2.0000, 21.1792,
         3.0000,  1.0000,  0.0000]): 1
576 - tensor([ 3.0000,  0.0000,  4.0000,  1.0000,  1.0000, 16.7000,  2.0000, 13.3029,
         1.0000,  2.0000,  0.0000]): 1
577 - tensor([ 1.0000,  1.0000, 40.0000,  0.0000,  0.0000, 27.7208,  0.0000, 87.5090,
         4.0000,  0.0000,  1.0000]): 0
578 - tensor([ 2.0000,  1.0000, 32.0000,  0.0000,  0.0000, 10.5000,  2.0000, 21.1792,
         2.0000,  0.0000,  1.0000]): 0
579 - tensor([ 3.0000,  1.0000, 29.0000,  0.0000,  0.0000,  7.2500,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
580 - tensor([ 3.0000,  1.0000, 16.0000,  4.0000,  1.0000, 39.6875,  2.0000, 13.3029,
         2.0000,  5.0000,  0.0000]): 0
581 - tensor([ 3.0000,  1.0000, 39.0000,  0.0000,  0.0000,  7.9250,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
582 - tensor([ 3.0000,  1.0000, 29.0000,  0.0000,  0.0000,  7.8958,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0


Epoch [1/100], Loss: 0.6291, Accuracy: 0.6690
Validation Loss: 0.6448, Validation Accuracy: 0.6798
Epoch [2/100], Loss: 0.5941, Accuracy: 0.6914
Validation Loss: 0.6323, Validation Accuracy: 0.6798
Epoch [3/100], Loss: 0.5761, Accuracy: 0.7195
Validation Loss: 0.6225, Validation Accuracy: 0.6798
Epoch [4/100], Loss: 0.5762, Accuracy: 0.7139
Validation Loss: 0.5786, Validation Accuracy: 0.6854
Epoch [5/100], Loss: 0.5621, Accuracy: 0.7307
Validation Loss: 0.5723, Validation Accuracy: 0.6854
Epoch [6/100], Loss: 0.5481, Accuracy: 0.7167
Validation Loss: 0.5767, Validation Accuracy: 0.6798
Epoch [7/100], Loss: 0.5304, Accuracy: 0.7363
Validation Loss: 0.5428, Validation Accuracy: 0.6854
Epoch [8/100], Loss: 0.5252, Accuracy: 0.7321
Validation Loss: 0.6531, Validation Accuracy: 0.6854
Epoch [9/100], Loss: 0.5385, Accuracy: 0.7321
Validation Loss: 0.5770, Validation Accuracy: 0.7135
Epoch [10/100], Loss: 0.5131, Accuracy: 0.7461
Validation Loss: 0.5545, Validation Accuracy: 0.6966
Epoch [11

Epoch [85/100], Loss: 0.3644, Accuracy: 0.8373
Validation Loss: 0.4477, Validation Accuracy: 0.8202
Epoch [86/100], Loss: 0.3732, Accuracy: 0.8359
Validation Loss: 0.4444, Validation Accuracy: 0.7921
Epoch [87/100], Loss: 0.3703, Accuracy: 0.8471
Validation Loss: 0.4633, Validation Accuracy: 0.8034
Epoch [88/100], Loss: 0.3672, Accuracy: 0.8373
Validation Loss: 0.4471, Validation Accuracy: 0.7978
Epoch [89/100], Loss: 0.3605, Accuracy: 0.8527
Validation Loss: 0.4477, Validation Accuracy: 0.8202
Epoch [90/100], Loss: 0.3684, Accuracy: 0.8415
Validation Loss: 0.4716, Validation Accuracy: 0.8034
Epoch [91/100], Loss: 0.3615, Accuracy: 0.8373
Validation Loss: 0.4625, Validation Accuracy: 0.8090
Epoch [92/100], Loss: 0.3588, Accuracy: 0.8485
Validation Loss: 0.4564, Validation Accuracy: 0.8090
Epoch [93/100], Loss: 0.3587, Accuracy: 0.8569
Validation Loss: 0.4684, Validation Accuracy: 0.7753
Epoch [94/100], Loss: 0.3763, Accuracy: 0.8485
Validation Loss: 0.5360, Validation Accuracy: 0.7921


train_accuracy,▁▃▃▃▄▅▅▅▆▆▆▆▇▇▆▆▇▆▇▇▇▇▇▇█▇▇█▇██▇▇▇██████
train_loss,█▇▆▅▅▅▄▄▄▄▄▃▃▃▃▃▂▃▂▂▂▂▂▂▂▁▂▂▂▁▂▁▂▂▁▁▁▂▁▁
val_accuracy,▁▁▁▁▃▂▄▅▆▄▆▆▆▇▆█▅▅▆▅▆▆▆▆█▆▆▆▆▇▇▅▇▅▇▇▇▆▇▆
val_loss,█▇▆█▅▆▄▃▃▄▂▂▅▂▂▂▄▁▁▁▂▄▂▁▁▂▁▁▂▁▄▁▂▂▂▂▂▄▂▅
train_accuracy,0.85133
train_loss,0.35214
val_accuracy,0.79213
val_loss,0.56828


Submission 파일 'submission.csv'가 생성되었습니다.


![](./IMG/leaderboard_RReLU.png)

## <후기>

A. 활성화함수 RReLU, Tanh, Sigmoid, ReLU, ReLU6, ELU, LeakyReLU 를 사용하여 학습을 하였다. 모든 학습은 단일 함수로만 시도하였고 그 중 RReLU 를 사용하였을 때 Kaggle에서 가장 높은 등수를 받게 되었다.

* RReLU
    * train_accuracy : 0.85133
    * val_accuracy : 0.79213
    
------
##### <정리>

- 경로를 나의 상황에 맞춰 설정하는 과정에서 
<b>'SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape'</b> 라는 오류가 났었다. 결과적으로 경로 사이에 \말고 /로 표시해줬어야 하는데 이것을 바로 파악하지 못하였고, 그래서 경로를 주피터노트북 안의 경로(http://localhost:8888/tree/deep_learning)로 설정을 하여 실행을 하였고 그래서 출력 부분에서 계속 오류가 났었다. 

#### Parsing(파싱)
- 텍스트나 데이터를 분석하고 원하는 정보를 추출하는 과정

#### epoch(에포크)
- 전체 학습 데이터 세트를 한 번 모델에 대해 순전파 및 역전파를 실행하는 것을 나타냄.
- <b>epoch</b>는 모델의 가중치 및 매개변수를 업데이트하고 학습 데이터에 대한 오차(손실)를 줄이는 데 사용됨

## model.eval()과 with torch.no_grad()에 대해

eval()과 torch.no_grad()가 무엇인지를 알아보던 도중 이 함수 둘 다 역전파를 비활성화 시키는 것을 알게 되었다. 그런데 eval()로 비활성화 시켜놓고 왜 또 torch.no_grad()로 코드를 감싸주는 건지 의문이 들어 이를 검색해보았다. 알게 된 내용을 설명으로 남긴다.
<br>
### 왜 model.eval()을 선언해놓고 또 with torch.no_grad():로 감싸주는거지?

#### with torch.no_grad()
* no_grad() with statement에 포함시키게 되면 Pytorch는 autograd engine을 꺼버린다. 이 말은 더 이상 자동으로 gradient를 트래킹하지 않는다는 말이 된다.
* torch.no_grad()의 주된 목적은 autograd를 끔으로써 메모리 사용량을 줄이고 연산 속도를 높히기 위함이다. 사실상 어짜피 안쓸 gradient인데 inference시에 굳이 계산할 필요가 없지 않은가?

#### model.eval()
* 그럼 torch.no_grad()만 쓰면 되지 않나? 맞는 말이지만, model.eval()의 역할은 약간 다르다. 
* 모델링 시 training과 inference시에 다르게 동작하는 layer들이 존재한다.
* 사실상 model.eval()는 이런 layer들의 동작을 inference(eval) mode로 바꿔준다는 목적으로 사용된다. 
* 따라서, 우리가 보통 원하는 모델의 동작을 위해서는 위의 두 가지를 모두 사용해야하는 것이 맞다.